In [ ]:
ENV_TYPE = "TEST"

if(ENV_TYPE != "TEST"):
  !git clone "https://github.com/kk-digital/kcg-ml-sd1p4.git"
  %cd kcg-ml-sd1p4
  !pip3 install -r requirements.txt
  exit()
  base_directory = "./"
else:
  base_directory = "../"

# Magical check for fixing all of our directory issues
import subprocess
output = subprocess.check_output(["pwd"], universal_newlines=True)
if "notebooks" in output:
    %cd ..
del output

In [ ]:
!python3 ./download_models.py

In [ ]:
!python3 ./process_models.py

In [ ]:
import subprocess

# Check if the package is installed
result = subprocess.run(['dpkg', '-s', 'libgl1-mesa-glx'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# If the package is not installed, install it
if 'is not installed and no information is available' in result.stderr:
    print("Installing libgl, which is needed to run the GA script.")
    !apt update
    !apt install libgl1-mesa-glx -y
else:
    print("Package 'libgl1-mesa-glx' is already installed.")


In [ ]:
!python3 scripts/prompts_ga.py --generations 2